In [48]:
import pickle
import numpy as np
from tqdm import tqdm
from functools import partial
import pandas as pd
from konlpy.tag import Komoran
import os
from gensim.models import Word2Vec
import json

In [43]:
preprocessed = pickle.load(open('preprocessed.pkl', 'rb'))

In [2]:
max_n = 5
total_data = [[], [], [], [], []]
for i in range(1, max_n + 1):
    file_name = 'data_256_cat%s.pkl' %i
    loaded_data = pickle.load(open(file_name, 'rb'))

    _image_data, _sentence_data, _sentence_tags, _hashtag_data, _metadata = loaded_data[1]

    _sentence_tags = np.array(_sentence_tags)
    _hashtag_data = np.array(_hashtag_data)
    
    total_data[0].append(_image_data)
    total_data[1].append(_sentence_data)
    total_data[2].append(_sentence_tags)
    total_data[3].append(_hashtag_data)
    total_data[4].append(_metadata)

In [3]:
_image_data = np.concatenate(total_data[0])
_sentence_data = np.concatenate(total_data[1])
_sentence_tags = np.concatenate(total_data[2])
_hashtag_data = np.concatenate(total_data[3])
_metadata = pd.concat(total_data[4])

In [4]:
np.random.seed(1234)
n_valid = 700
n_train = _image_data.shape[0] - n_valid
n_total = n_train + n_valid
indices = np.random.choice(np.arange(_image_data.shape[0]), n_total, replace=False)
train_idx = indices[:n_train]
valid_idx = indices[n_train:]

In [5]:
image_data = _image_data[train_idx]
sentence_data = _sentence_data[train_idx]
sentence_tags = _sentence_tags[train_idx]
hashtag_data = _hashtag_data[train_idx]
metadata = _metadata.iloc[train_idx]

In [6]:
image_valid = _image_data[valid_idx]
sentence_valid = _sentence_data[valid_idx]
sentence_tags_valid = _sentence_tags[valid_idx]
hashtag_valid = _hashtag_data[valid_idx]
metadata_valid = _metadata.iloc[valid_idx]

In [7]:
print(image_data.shape)
print(sentence_data.shape)
print(sentence_tags.shape)
print(hashtag_data.shape)
print(metadata.shape)

(34300, 256, 256, 3)
(34300,)
(34300,)
(34300,)
(34300, 15)


In [49]:
all_hashtag = []
for tags in tqdm(preprocessed['hashtag']):
    _hash = tags.replace("'", '"')
    pos = _hash.find('\\U')
    while pos != -1:
        _hash = _hash.replace(_hash[pos:pos+10], chr(int(_hash[pos+2:pos+10], 16)))
        pos = _hash.find('\\U')
    all_hashtag.append(json.loads(_hash))

100%|██████████| 867737/867737 [00:03<00:00, 246300.12it/s]


In [51]:
all_tags = []

for tags in all_hashtag:
    all_tags.extend(tags)

In [84]:
word_df = pd.DataFrame({'word':all_tags})
filtered_df = word_df.value_counts()

use_hashtag = filtered_df[((filtered_df<len(all_hashtag)*0.01) & (filtered_df>len(all_hashtag)*0.0001))].index
use_hashtag = [t[0] for t in use_hashtag]
use_hashtag = sorted(use_hashtag)
tag_dict = {t:i for i, t in enumerate(use_hashtag)}

n_tags = len(use_hashtag)

In [ ]:
y_hashtag = np.zeros((hashtag_data.shape[0], n_tags), dtype='float32')

for i, tags in enumerate(hashtag_data):
    for tag in tags:
        if tag in tag_dict:
            y_hashtag[i, tag_dict[tag]] = 1.
            
valid_hashtag = np.zeros((hashtag_valid.shape[0], n_tags), dtype='float32')
for i, tags in enumerate(hashtag_valid):
    for tag in tags:
        if tag in tag_dict:
            valid_hashtag[i, tag_dict[tag]] = 1.

In [100]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [101]:
tf.reset_default_graph()

learning_rate1 = 0.001
learning_rate2 = 0.001
n_hidden = 128

inputs = tf.placeholder(tf.float32, shape=[None, 256, 256, 3])
with tf.variable_scope('auto_encoder'):
    net = tf.layers.conv2d(inputs, 32, [5,5], strides=2, padding='SAME')
    net = tf.layers.conv2d(net, 8, [5,5], strides=2, padding='SAME')
    middle = tf.layers.conv2d(net, 4, [5,5], strides=2, padding='SAME')
    net = tf.layers.conv2d_transpose(middle, 8, [5,5], strides=2, padding='SAME')
    net = tf.layers.conv2d_transpose(net, 32, [5,5], strides=2, padding='SAME')
    outputs = tf.layers.conv2d_transpose(net, 3, [5,5], strides=2, padding='SAME')

loss = tf.reduce_mean(tf.square(outputs - inputs))
train_op = tf.train.AdamOptimizer(learning_rate1).minimize(loss)

In [102]:
middle

<tf.Tensor 'auto_encoder/conv2d_2/BiasAdd:0' shape=(?, 32, 32, 4) dtype=float32>

In [103]:
flatten = tf.layers.flatten(middle)

with tf.variable_scope('hashtag'):
    tag_output = tf.placeholder(tf.float32, shape=(None, n_tags))
    hidden1 = tf.layers.dense(flatten, 256, activation=tf.nn.sigmoid)
    hidden2 = tf.layers.dense(hidden1, 256, activation=tf.nn.sigmoid)
    hidden3 = tf.layers.dense(hidden2, 256, activation=tf.nn.sigmoid)
    outputs = tf.layers.dense(hidden3, n_tags, activation=None)

loss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = outputs, labels = tag_output))
var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='hashtag')
train_op2 = tf.train.AdamOptimizer(learning_rate2).minimize(loss2, var_list=var_list)
train_op2_fine = tf.train.AdamOptimizer(learning_rate2).minimize(loss2)

In [104]:
with tf.device('/gpu:0'):
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, 
                                            log_device_placement=True))
    sess.run(tf.global_variables_initializer())

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:68:00.0, compute capability: 7.5



In [105]:
batch_size = 700
n_epoch = 20
n_batch = n_train // batch_size

for epoch in range(n_epoch):
    total_loss = 0
    for batch in tqdm(range(n_batch)):
        batch_img = image_data[batch*batch_size:(batch+1)*batch_size]
        _, c = sess.run([train_op, loss], feed_dict={inputs:batch_img})
        total_loss += c
        
    c = sess.run(loss, feed_dict={inputs:image_valid})
    
    print('[Epoch: %s] cost = %.3f, valid cost = %.3f' %(epoch+1, total_loss / n_batch, c))

  0%|          | 0/49 [00:00<?, ?it/s]

[Epoch: 1] cost = 5393.984, valid cost = 1712.024


  0%|          | 0/49 [00:00<?, ?it/s]

[Epoch: 2] cost = 1346.154, valid cost = 1253.117


  0%|          | 0/49 [00:00<?, ?it/s]

...

[Epoch: 18] cost = 449.623, valid cost = 458.071


  0%|          | 0/49 [00:00<?, ?it/s]

[Epoch: 19] cost = 428.101, valid cost = 451.137


100%|██████████| 49/49 [00:35<00:00,  1.37it/s]


[Epoch: 20] cost = 421.833, valid cost = 444.888


In [106]:
batch_size = 700
n_epoch = 50
n_batch = n_train // batch_size

for epoch in range(n_epoch):
    total_loss = 0
    for batch in tqdm(range(n_batch)):
        batch_img = image_data[batch*batch_size:(batch+1)*batch_size]
        batch_tag_out = y_hashtag[batch*batch_size:(batch+1)*batch_size]
        _, c = sess.run([train_op2, loss2], feed_dict={inputs:batch_img, 
                                                       tag_output:batch_tag_out})
        total_loss += c
    
    c = sess.run(loss2, feed_dict={inputs:image_valid, 
                                   tag_output:valid_hashtag})
    print('[Epoch %s] cost = %.3f, valid cost = %.3f' %(epoch+1, total_loss / n_batch, c))

100%|██████████| 49/49 [00:23<00:00,  2.12it/s]


NameError: name 'valid_hashtag' is not defined

In [ ]:
batch_size = 700
n_epoch = 50
n_batch = n_train // batch_size

for epoch in range(n_epoch):
    total_loss = 0
    for batch in tqdm(range(n_batch)):
        batch_img = image_data[batch*batch_size:(batch+1)*batch_size]
        batch_tag_out = y_hashtag[batch*batch_size:(batch+1)*batch_size]
        _, c = sess.run([train_op2_fine, loss2], feed_dict={inputs:batch_img, 
                                                       tag_output:batch_tag_out})
        total_loss += c
    
    c = sess.run(loss2, feed_dict={inputs:image_valid, 
                                   tag_output:valid_hashtag})
    print('[Epoch %s] cost = %.3f, valid cost = %.3f' %(epoch+1, total_loss / n_batch, c))

In [158]:
saver = tf.train.Saver()
saver.save(sess, './ver16')

'./ver15'

In [159]:
saver = tf.train.import_meta_graph('ver15.meta')

saver.restore(sess, tf.train.latest_checkpoint('./'))

INFO:tensorflow:Restoring parameters from ./ver15


In [ ]:
    c = sess.run('dense/BiasAdd:0', feed_dict={inputs:image_valid, 
                                   dec_input:dec_input_valid,
                                   dec_output:dec_output_valid})

In [171]:
whos

Variable              Type                    Data/Info
-------------------------------------------------------
Komoran               type                    <class 'konlpy.tag._komoran.Komoran'>
Word2Vec              type                    <class 'gensim.models.word2vec.Word2Vec'>
batch                 int                     18
batch_dec_in          ndarray                 700x256x1865: 334208000 elems, type `float64`, 2673664000 bytes (2549.8046875 Mb)
batch_dec_out         ndarray                 700x256: 179200 elems, type `float64`, 1433600 bytes (1.3671875 Mb)
batch_img             ndarray                 700x256x256x3: 137625600 elems, type `uint8`, 137625600 bytes (131.25 Mb)
batch_size            int                     700
c                     float32                 2.3351858
dec_cell              DropoutWrapper          <tensorflow.python.keras.<...>object at 0x7f66fe4a5890>
dec_in_data           ndarray                 13300x256x1864: 6346547200 elems, type `float64`, 5

In [163]:
sess.run('dense/BiasAdd:0', feed_dict={''})

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,256,256,3]
	 [[node Placeholder (defined at <ipython-input-138-328e441e7177>:7) ]]
	 [[dense/BiasAdd/_887]]
  (1) Invalid argument: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,256,256,3]
	 [[node Placeholder (defined at <ipython-input-138-328e441e7177>:7) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'Placeholder':
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/svclaw2000/venv_capstone/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/svclaw2000/venv_capstone/lib/python3.7/site-packages/traitlets/config/application.py", line 845, in launc

In [166]:
def translate(image):
    dec_inp = np.zeros(shape=(1, max_seq_length, n_chars), dtype='float32')
    dec_inp[0,0,output_dict['\t']] = 1.
    result = sess.run('dense/BiasAdd:0', 
                      feed_dict={inputs: [image],
                                 dec_input: dec_inp})
    result = np.argmax(result, axis=2)
    decoded = [output_chars[i] for i in result[0]]
    end = decoded.index('\n') if '\n' in decoded else len(decoded)
    translated = ''.join(decoded[:end])
    return translated

In [172]:
pickle.dump([
        max_seq_length,
        n_chars,
        output_dict,
        output_chars
], open('seqlen_nchar_outdict_outchar.pkl', 'wb'))

In [170]:
tf.__version__

'2.3.1'

In [167]:
idx = 90
print(sentence_data[idx])
translate(image_data[idx])

	버버리 카톡문의 카드결제 가능 정품과 헷갈릴정도의 최상급만 취급 제품 출고영상 확인가능해요 배송기간 일 잡아주세요 풀구성 배송 피드에 없는 제품도 찾아드려요 제품문의는 프로필 상단 링크로 꾸욱



'오이     \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t'

In [79]:
from PIL import Image
import matplotlib.pyplot as plt

In [98]:
def translate(image):
    prediction = tf.argmax(outputs2, 2)
    dec_inp = np.zeros(shape=(1, max_seq_length, dict_len), dtype='float32')
    dec_inp[0,:,output_dict['\t']] = 1.
    # dec_out = np.eye(dict_len)[dec_out]
    result = sess.run(prediction, 
                      feed_dict={inputs: [image],
                                 dec_inputs: dec_inp})
    decoded = [output_chars[i] for i in result[0]]
    end = decoded.index('\n') if '\n' in decoded else len(decoded)
    translated = ''.join(decoded[:end])
    return translated

In [103]:
translate(image_data[50])

'오<UNK><UNK>'

In [80]:
start_idx, end_idx = 50, 100

results = sess.run(outputs2, feed_dict={inputs:image_data[start_idx:end_idx]})
print('정답 예측')

for l, p, img in zip(y_train[start_idx:end_idx], l_label[np.argmax(results, axis=1)], image_data[start_idx:end_idx]):
    print([l==p, l, p])
    plt.imshow(img)
    plt.show()

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: You must feed a value for placeholder tensor 'Placeholder_1' with dtype float and shape [?,256,2069]
	 [[node Placeholder_1 (defined at <ipython-input-70-42deb1a72a5c>:19) ]]
	 [[generation_dec/dense/BiasAdd/_167]]
  (1) Invalid argument: You must feed a value for placeholder tensor 'Placeholder_1' with dtype float and shape [?,256,2069]
	 [[node Placeholder_1 (defined at <ipython-input-70-42deb1a72a5c>:19) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'Placeholder_1':
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/svclaw2000/venv_capstone/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/svclaw2000/venv_capstone/lib/python3.7/site-packages/traitlets/config/application.

In [52]:
start_idx, end_idx = 0, 50

results = sess.run(outputs2, feed_dict={inputs:image_valid[start_idx:end_idx]})
print('정답 예측')
[[l==p, l, p] for l, p in zip(y_train[start_idx:end_idx], l_label[np.argmax(results, axis=1)])]

정답 예측


[[False, '애완동물', '여행'],
 [False, '일상', '여행'],
 [False, '애완동물', '여행'],
 [False, '셀카', '패션'],
 [False, '여행', '셀카'],
 [False, '여행', '연애'],
 [False, '애완동물', '음식'],
 [False, '음식', '여행'],
 [False, '패션', '여행'],
 [True, '애완동물', '애완동물'],
 [False, '일상', '애완동물'],
 [False, '여행', '애완동물'],
 [False, '패션', '연애'],
 [True, '패션', '패션'],
 [False, '음식', '여행'],
 [False, '여행', '음식'],
 [False, '음식', '여행'],
 [False, '패션', '애완동물'],
 [False, '음식', '셀카'],
 [False, '애완동물', '연애'],
 [False, '일상', '음식'],
 [True, '여행', '여행'],
 [False, '음식', '패션'],
 [False, '연애', '셀카'],
 [False, '셀카', '음식'],
 [False, '음식', '패션'],
 [False, '애완동물', '연애'],
 [False, '패션', '일상'],
 [False, '셀카', '애완동물'],
 [False, '일상', '패션'],
 [False, '음식', '여행'],
 [True, '패션', '패션'],
 [False, '패션', '일상'],
 [False, '일상', '연애'],
 [True, '연애', '연애'],
 [False, '애완동물', '패션'],
 [False, '일상', '연애'],
 [False, '애완동물', '패션'],
 [False, '애완동물', '패션'],
 [False, '연애', '여행'],
 [False, '셀카', '여행'],
 [True, '음식', '음식'],
 [False, '여행', '애완동물'],
 [True, '애완동물', '애완동물'],
 [Fal